In [2]:
import torch
from torch import nn

def corr2d(x,k):
    '''计算二维互相关运算'''
    #获取卷积核的高（h）和宽（w）（行数与列数）
    h,w = k.shape
    #输出计算结果y的形状为[输入张量x的行数-h+1，输入张量x的列数-w+1]，同时初始化y的数值（全部为0）
    y = torch.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))
    #嵌套两层循环计算y对应位置的值并更新
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            #对应位置的值为x取i,j为起点的与卷积核同样形状的子张量，计算与卷积核的点积并累加
            y[i,j] = (x[i:i+h,j:j+w]*k).sum()

    return y

#验证上述二维互相关运算的输出
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
print(corr2d(X,K))

tensor([[19., 25.],
        [37., 43.]])


In [3]:
#实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [4]:
#卷积层的一个简单应用：检测图像中不同颜色的边缘
X = torch.ones((6,8))
X[:,2:6] = 0
print(X)

K = torch.tensor([[1.0,-1.0]])
Y = corr2d(X,K)
print(Y)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [11]:
#学习由x生成y的卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i+1)%2 == 0:
        print(f'batch {i+1},loss {l.sum():.3f}')

#所学的卷积核的权重张量
print(conv2d.weight.data.reshape((1,2)))

batch 2,loss 5.383
batch 4,loss 1.487
batch 6,loss 0.488
batch 8,loss 0.180
batch 10,loss 0.070
tensor([[ 1.0180, -0.9641]])
